In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
import tensorflow as tf
import seaborn as sns

sns.set()

tf.compat.v1.random.set_random_seed(1234)

In [ ]:
from Model import Model, predict_next_day, forecast, stock_price_preprocessing
from Pre import generate_benchmark_data, calculate_excess_return, discretize_features, cfs, generate_composite_index


In [ ]:
stock_file_path = "./000001.SZ.csv"

stock_data = pd.read_csv(stock_file_path)

stock_data = generate_benchmark_data(stock_data)

stock_data = calculate_excess_return(stock_data)

stock_data = generate_composite_index(stock_data)

target_column = 'excess_return'  
    
feature_columns = stock_data.columns.tolist()

feature_columns.remove(target_column)

stock_data = discretize_features(stock_data, feature_columns)

selected_features = cfs(stock_data, target_column)

print("Selected Features:", selected_features)

In [ ]:
file_folder_path = ".\stock data"
other_folder_path = ".\CG2408_PHASE2"
other_other_folder_path = ".\CG2408_PHASE3"

csv_files = [f for f in os.listdir(file_folder_path) if f.endswith('.csv')]

for csv_file in csv_files:
    file_path = os.path.join(file_folder_path, csv_file)
    other_data_file = os.path.join(other_folder_path, csv_file)
    other_other_data_file = os.path.join(other_other_folder_path, csv_file)
    
    file_name_without_extension = os.path.splitext(csv_file)[0]

    data = pd.read_csv(file_path)
    other_data = pd.read_csv(other_data_file)
    other_other_data =  pd.read_csv(other_other_data_file)

    merged_data = pd.concat([other_data,data], axis=0).reset_index(drop=True)

    merged_data = generate_composite_index(merged_data)
    
    threshold = 0.1
    
    merged_data['price_change'] = merged_data['close'].pct_change()

    alerts = merged_data[abs(merged_data['price_change']) > threshold]


    first_alert_date = 1000
    if not alerts.empty:
        if alerts.index[0]>1000:
            pass
        elif alerts.index[0]<250:
            first_alert_date=250 
        else:
            first_alert_date = alerts.index[0]
    merged_data = merged_data[merged_data.index <= first_alert_date]
    
    merged_data = pd.concat([other_data, merged_data])

    merged_data = merged_data.drop_duplicates()

    merged_data = merged_data.reset_index(drop=True)

    merged_data= merged_data.iloc[::-1][selected_columns]

    X, y = stock_price_preprocessing(merged_data, 1, 1, selected_columns)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.1)

    num_samples = len(X_train)

    num_layers = 1
    size_layer = 128
    size = X.shape[2]
    timestamp = 5
    epoch = 300
    dropout_rate = 0.8
    learning_rate = 0.001
    batch_size = 64

    modelnn, y_test, y_pred = forecast(X_train, y_train, X_test, y_test, learning_rate, 
                                       num_layers, size, size_layer, dropout_rate, 
                                       epoch, batch_size)

    historical_data = X[-1][0]
    next_day_price = predict_next_day(modelnn, historical_data)
    close_value = merged_data['close'].iloc[-1]
    relative_change = (next_day_price - close_value) / close_value
    print(f"Relative change for {csv_file}: {relative_change}")
    